In [72]:
import pandas as pd
import numpy as np

#csv1 = pd.read_csv("results_div_resume_training_399880.csv") # Bias Bigg
csv1=pd.read_csv('ensemble_results (1).csv')
csv2 = pd.read_csv("results_cpk286_6.csv") #Bias High
#csv3 = pd.read_csv("results_rna3535_110K.csv")  # 105+5
csv3 = pd.read_csv('results_cpk286_8_litteboy.csv')
csv4 = pd.read_csv("ensemble_results_lil_boy_6.csv") 
#csv5 = pd.read_csv("res_new_rna3535_100K.csv") #  UNBIAS

weights = {
    "model1": 0.85797,  
    "model2": 0.85113,  
    "model3": 0.85334, 
    "model4": 0.86280,
}

dfs = [csv1, csv2, csv3, csv4]

merged = dfs[0][['ID']]  

for i, df in enumerate(dfs):
    merged = merged.merge(df[['ID', 'is_correct']], on='ID', suffixes=('', f'{i+1}'))

merged.head()

,ID,is_correct,is_correct2,is_correct3,is_correct4
0,0,True,True,False,True
1,1,False,False,False,False
2,2,True,True,True,True
3,3,False,False,True,False
4,4,False,False,True,False


In [73]:
merged.rename(columns={'is_correct': 'is_correct1'}, inplace=True)

In [74]:
for i in range(len(dfs)):
    merged[f'is_correct{i+1}'] = merged[f'is_correct{i+1}'].map({True: 1, False: 0})
merged.head()

,ID,is_correct1,is_correct2,is_correct3,is_correct4
0,0,1,1,0,1
1,1,0,0,0,0
2,2,1,1,1,1
3,3,0,0,1,0
4,4,0,0,1,0


In [75]:
merged['weighted_sum'] = sum(
    weights[f'model{i+1}'] * merged[f'is_correct{i+1}'] for i in range(len(dfs))
)

total_weight = sum(weights.values())
merged['is_correct'] = (merged['weighted_sum'] >= (total_weight / 2)).astype(bool)

merged[['ID', 'is_correct']].to_csv("ensemble_results_final.csv", index=False)

merged


,ID,is_correct1,is_correct2,is_correct3,is_correct4,weighted_sum,is_correct
0,0,1,1,0,1,2.57190,True
1,1,0,0,0,0,0.00000,False
2,2,1,1,1,1,3.42524,True
3,3,0,0,1,0,0.85334,False
4,4,0,0,1,0,0.85334,False
...,...,...,...,...,...,...,...
9995,9995,1,1,1,1,3.42524,True
9996,9996,0,0,0,0,0.00000,False
9997,9997,1,1,1,1,3.42524,True
9998,9998,0,0,0,0,0.00000,False


In [76]:
print(f'{len(merged[merged['is_correct']==True])/len(merged)*100:.2f}%')

33.28%


In [77]:
print(f'{len(merged[merged['is_correct']==False])/len(merged)*100:.2f}%')

66.72%
